Dataset info:
All 50 states except Delaware (idk why), Puerto Rico, U.S. Pacific Trust Territories and Possessions, 

In [45]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [46]:
jan = pd.read_csv('data/data_by_month/jan_2023.csv')
feb = pd.read_csv('data/data_by_month/feb_2023.csv')
mar = pd.read_csv('data/data_by_month/mar_2023.csv')
apr = pd.read_csv('data/data_by_month/apr_2023.csv')
may = pd.read_csv('data/data_by_month/may_2023.csv')
jun = pd.read_csv('data/data_by_month/jun_2023.csv')
jul = pd.read_csv('data/data_by_month/jul_2023.csv')
aug = pd.read_csv('data/data_by_month/aug_2023.csv')
sep = pd.read_csv('data/data_by_month/sep_2023.csv')
oct = pd.read_csv('data/data_by_month/oct_2023.csv')
nov = pd.read_csv('data/data_by_month/nov_2023.csv')
dec = pd.read_csv('data/data_by_month/dec_2023.csv')
df = pd.concat([jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec])
ids = pd.read_csv('data/airline_id.csv')

In [ ]:
# data cleaning
df.fillna(0, inplace=True)

# rename airlines
airline_names = pd.DataFrame(df['OP_CARRIER_AIRLINE_ID'].unique(), columns=['Code']).merge(ids, on='Code', how='left')
airline_names['Description'] = airline_names['Description'].apply(lambda x: x.split(':')[0])
airline_names['Description'] = airline_names['Description'].apply(lambda x: x.split('LLC')[0])
airline_names['Description'] = airline_names['Description'].apply(lambda x: x.split('Inc.')[0])
airline_names['Description'] = airline_names['Description'].apply(lambda x: x.split('Corp')[0])
airline_names['Description'] = airline_names['Description'].apply(lambda x: x.split('Co.')[0])

df


In [ ]:
# create an array of the delay types
delay_fields = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'CANCELLED']

# prep the df to support a total flight field per airline
df['TOTAL_FLIGHTS'] = 1

# group by airline and sum the desired fields, merge with airline names
grouped_df = df.groupby('OP_CARRIER_AIRLINE_ID').agg({delay_fields[0]: 'sum', delay_fields[1]: 'sum', delay_fields[2]: 'sum', delay_fields[3]: 'sum', delay_fields[4]: 'sum', 'CANCELLED': 'sum', 'DIVERTED': 'sum', 'TOTAL_FLIGHTS': 'sum'}).reset_index()
grouped_with_airline = pd.merge(grouped_df, airline_names, left_on='OP_CARRIER_AIRLINE_ID', right_on='Code')


average_delays = grouped_with_airline
for delay in delay_fields:
    average_delays[delay] = grouped_with_airline[delay] / grouped_with_airline['TOTAL_FLIGHTS']

Look at averages

In [ ]:

for delay in delay_fields:
    average_delays_sorted = average_delays.sort_values(f'{delay}', ascending=False)
    plt.figure()
    plt.title(f'Average {delay} by Airline')
    plt.xlabel('Airline')
    plt.ylabel(f'Average {delay}')
    plt.xticks(rotation=90)
    sns.barplot(x='Description', y=f'{delay}', data=average_delays_sorted)





Create a ranking for best and worst airlines

In [ ]:
# create weighted scores for each airline
ranks = pd.DataFrame(zip(airline_names['Description'], np.zeros(len(airline_names['Description']))), columns=['Airline', 'Score'])
for feature in delay_fields:
    temp = average_delays.sort_values(feature, ascending=False)
    position = 1
    for row in temp.iterrows():
        ranks.loc[ranks['Airline'] == row[1]['Description'], 'Score'] += position
        position += 1
sorted_ranks = ranks.sort_values('Score', ascending=False)

# add a low budget flag (according to wikipedia)
low_budget = ['Allegiant Air', 'Frontier Airlines ', 'JetBlue Airways', 'Southwest Airlines ', 'Spirit Air Lines']
sorted_ranks['Low Budget'] = sorted_ranks['Airline'].apply(lambda x: 1 if x in low_budget else 0)


plt.figure()
plt.title('Airline Ranking')
plt.xlabel('Airline')
plt.ylabel('Score')
plt.xticks(rotation=90)
sns.barplot(x='Airline', y='Score', data=sorted_ranks, hue='Low Budget')

        

It makes sense that JetBlue is last, multiple sites rank it as the worst performer in regards to delays and other factors

Look into weather

In [ ]:
df
weather_delays = df.groupby('ORIGIN_STATE_NM').agg({'WEATHER_DELAY': 'sum', 'TOTAL_FLIGHTS': 'sum'}).reset_index()
weather_delays['AVG'] = weather_delays['WEATHER_DELAY'] / weather_delays['TOTAL_FLIGHTS']
weather_delays_sorted = weather_delays.sort_values('AVG', ascending=False)

# rename the territories
weather_delays_sorted['ORIGIN_STATE_NM'] = weather_delays_sorted['ORIGIN_STATE_NM'].apply(lambda x: 'U.S. Territories' if x == 'U.S. Pacific Trust Territories and Possessions' else x)

plt.figure()
plt.title('Average Weather Delay by State')
plt.xlabel('State')
plt.ylabel('Average Weather Delay')
plt.xticks(rotation=90, fontsize=9)
sns.barplot(x='ORIGIN_STATE_NM', y='AVG', data=weather_delays_sorted)



Look into which state flies which airline the most, see if there is a pattern? See if a top airline for a state relates to weather delays, carrier delays, etc

In [ ]:
# Group the data by state and airline, and get the count of flights
flight_counts = df.groupby(['ORIGIN_STATE_ABR', 'OP_CARRIER_AIRLINE_ID']).size().reset_index(name='Flight Count')

flight_counts_airlines = pd.merge(flight_counts, airline_names, left_on='OP_CARRIER_AIRLINE_ID', right_on='Code')
flight_counts_airlines.drop(columns=['OP_CARRIER_AIRLINE_ID', 'Code'], inplace=True)
states = flight_counts_airlines['ORIGIN_STATE_ABR'].unique()

top_airlines = {}
for state in states:
    state_flights = flight_counts_airlines[flight_counts_airlines['ORIGIN_STATE_ABR'] == state]
    state_flights_sorted = state_flights.sort_values('Flight Count', ascending=False)
    top_airlines[state] = state_flights_sorted.head(1)['Description'].values[0]
df_top_airlines = pd.DataFrame(top_airlines.items(), columns=['State', 'Airline'])

fig = px.choropleth(df_top_airlines, locations='State', locationmode='USA-states', color='Airline', scope='usa', title='Top Airline by State')
fig.show()

# TODO: Look into Allegiant Airlines to see how common their flights are in the northwest